In [1]:
import folium
import branca.colormap as cm
import clustering as cl
import networkx as nx
import spatial as sp

In [2]:
metro = nx.read_gml("data/graph/metro.gml")
bus = nx.Graph(nx.read_gml("data/graph/bus.gml").to_undirected())
tram = nx.Graph(nx.read_gml("data/graph/tram.gml").to_undirected())

metro_nodes = list(metro.nodes(data = True))
bus_nodes = list(bus.nodes(data = True))
tram_nodes = list(tram.nodes(data = True))

metro_regions, _ = cl.cluster_stops(metro_nodes, 2)

In [3]:
metro_points = [x["centroid"] for _, x in metro_regions.items()]
metro_region_neighbors = cl.get_neighbor_list(metro_points, radius=5)

In [4]:
metro_neighbor_graph = nx.Graph(metro_region_neighbors)

In [5]:
m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(metro_points)[0], min(metro_points)[1]], [max(metro_points)[0], max(metro_points)[1]]])

for n in metro_neighbor_graph:
    metro_neighbor_graph.nodes[n]["centroid"] = metro_regions[n]["centroid"]
    metro_neighbor_graph.nodes[n]["members"] = metro_regions[n]["members"]
    
    folium.Circle(
        radius = 150,
        location = metro_neighbor_graph.nodes[n]["centroid"],
        popup = "<b>" + str(n) + "</b>" + "<br>" + ", ".join(metro_neighbor_graph.nodes[n]["members"]),
        color = 'blue',
        fill = True,
        fill_color='blue',
        fill_opacity = 0.9
    ).add_to(m)
    
for e in metro_neighbor_graph.to_directed().edges:
    metro_edge_coords = [metro_neighbor_graph.nodes[e[0]]["centroid"], metro_neighbor_graph.nodes[e[1]]["centroid"]]
    folium.PolyLine(metro_edge_coords, color = "blue", weight = 1.3).add_to(m)

In [6]:
linear = cm.linear.Dark2_07.scale(0, len(metro_regions)).to_step(len(metro_regions))
linear.caption = "Regions"
m.add_child(linear)

for key, val in metro_regions.items():
    for c, elem in enumerate(val["members"]):

        folium.Circle(
            radius = 100,
            location = val["coords"][c],
            popup = "Region: " + str(key) + "<br>" + elem,
            color = linear(key),
            fill = True,
            fill_color = linear(key),
            fill_opacity=0.3,
        ).add_to(m)
m

In [7]:
# Bus and tram share some stops, some processing is needed before uniting the nets. In the composition, bus attributes take the precedence
bus_tram = nx.compose(tram, bus)

for node in tram.nodes(data = True):
    if node[1]["routes"] != bus_tram.node[node[0]]["routes"]:
        bus_tram.node[node[0]]["routes"] += "," + (node[1]["routes"])

net = nx.union(metro, bus_tram)
stops = list(net.nodes(data = True))
regions, stops_mapping = cl.cluster_stops(stops, 2)

points = [x["centroid"] for _, x in regions.items()]
region_neighbors = cl.get_neighbor_list(points, radius=5)

neighbor_graph = nx.Graph(region_neighbors)

In [8]:
m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(points)[0], min(points)[1]], [max(points)[0], max(points)[1]]])

for n in neighbor_graph:
    neighbor_graph.nodes[n]["centroid"] = regions[n]["centroid"]
    neighbor_graph.nodes[n]["members"] = regions[n]["members"]
    
    folium.Circle(
        radius = 150,
        location = neighbor_graph.nodes[n]["centroid"],
        popup = "<b>" + str(n) + "</b>" + "<br>" + ", ".join(neighbor_graph.nodes[n]["members"]),
        color = 'blue',
        fill = True,
        fill_color='blue',
        fill_opacity = 0.9
    ).add_to(m)
    
for e in neighbor_graph.to_directed().edges:
    edge_coords = [neighbor_graph.nodes[e[0]]["centroid"], neighbor_graph.nodes[e[1]]["centroid"]]
    folium.PolyLine(edge_coords, color = "blue", weight = 1.3).add_to(m)
    
m

In [17]:
c2c_graph = cl.c2c_flow_graph(net, stops_mapping)

In [37]:
m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(points)[0], min(points)[1]], [max(points)[0], max(points)[1]]])

max_weight = max(dict(c2c_graph.edges).items(), key=lambda x: x[1]['weight'])[1]["weight"]

linear = cm.linear.RdYlBu_08
linear.caption = "Weight"
m.add_child(linear)

for e in c2c_graph.to_directed().edges(data = True):
    weight = c2c_graph[e[0]][e[1]]["weight"]
    
    edge_coords = [regions[e[0]]["centroid"], regions[e[1]]["centroid"]]
    folium.PolyLine(
        edge_coords,
        color = linear(weight/max_weight),
        weight = 3 * weight/max_weight * 5,
        tooltip = "<b>Weight: " + str(weight) + "</b>" + "<br>"
       ).add_to(m)

for n in c2c_graph:
    
    folium.Circle(
        radius = 150,
        location = regions[n]["centroid"],
        popup = "<b>Region: " + str(n) + "</b>" + "<br>",
        color = 'blue',
        fill = True,
        fill_color='blue',
        fill_opacity = 0.9
    ).add_to(m)

m